In [13]:
import os
import numpy as np
import scipy
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import matplotlib.pyplot as plt
import math
import librosa

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
import warnings
from sklearn.exceptions import DataConversionWarning
from sklearn.model_selection import RandomizedSearchCV

# kNN classifier code

In [ ]:
#Importing the required modules
import numpy as np
from scipy.stats import mode
 
#Euclidean Distance
def eucledian(p1,p2):
    dist = np.sqrt(np.sum((p1-p2)**2))
    return dist
 
#Function to calculate KNN
def predict(x_train, y , x_input, k):
    op_labels = []
    #Loop through the Datapoints to be classified
    for item in x_input: 
        #Array to store distances
        #point_dist = []
        point_dist=np.zeros(x_input.shape[0])
        #Loop through each training Data
        for j in range(len(x_train)): 
            distances = eucledian(np.array(x_train[j,:]) , item)#distance.euclidean()# 
            #Calculating the distance
            point_dist.append(distances) 
        point_dist = np.array(point_dist) 
        #Sorting the array while preserving the index, and keeping the first K datapoints
        dist = np.argsort(point_dist)[:k] 
        #Labels of the K datapoints from above
        labels = y[dist]
        #Majority voting using mode()
        lab = mode(labels) 
        lab = lab.mode[0]
        op_labels.append(lab)
 
    return op_labels

# Feature Extraction

## Vggish

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import librosa
vggmodel = hub.load('https://tfhub.dev/google/vggish/1')

def embedding_from_fn(fn):
    x, sr = librosa.load(fn,sr=None)
    x_16k = librosa.resample(x,sr,16000) #resample to 16KHz
    embedding = np.array(vggmodel(x_16k))
    return embedding

## Info csv load

In [16]:
relevant_genres = ['funk', 'hiphop', 'jazz', 'rock']

In [14]:
from scipy.io.wavfile import read as wavread
def ToolReadAudio(cAudioFilePath):    
    [samplerate, x] = wavread(cAudioFilePath)    
    if x.dtype == 'float32':        
        audio = x    
    else:        
        # change range to [-1,1)        
        if x.dtype == 'uint8':            
            nbits = 8        
        elif x.dtype == 'int16':            
            nbits = 16        
        elif x.dtype == 'int32':            
            nbits = 32        
        audio = x / float(2**(nbits - 1))    
        # special case of unsigned format    
    if x.dtype == 'uint8':        
        audio = audio - 1.    
    return (samplerate, audio)

In [17]:
import pandas as pd
info = pd.read_csv('./groove/info.csv')
#df=info[info['style'].isin(relevant_genres)]

#df.pivot(index='style', columns='beat_type', values='audio_filename')
df = pd.pivot_table(info, values='audio_filename', index=['style'],
                    columns=['beat_type'], aggfunc='count')
#df[df['beat'] >= 20]

info = info[info['style'].isin(relevant_genres)]
info = info[info['beat_type'] == "beat"]
info = info[info['audio_filename'].notna()].reset_index()
#locs = info['audio_filename'].to_numpy()


## Loading and Normalizing (and converting to Mono) Audio

In [49]:
info['audio_filename'][0]

'drummer1/session1/1_funk_80_beat_4-4.wav'

In [51]:
sr,audio = ToolReadAudio('./groove/'+'drummer1/session1/1_funk_80_beat_4-4.wav')
audio.shape[-1]

<ipython-input-3-70eaf167f057>:3: WavFileWarning: Chunk (non-data) not understood, skipping it.
  [samplerate, x] = wavread(cAudioFilePath)


2

# Feature Extraction and Pre-Processing

In [18]:
info['audio'] = ''
for index, row in info.iterrows():
    #if index % 10 == 0:
        #print(index)
    try:
        sr,audio = ToolReadAudio('./groove/'+row['audio_filename'])
        if audio.shape[-1] == 2:
            info['audio'][index] = np.mean(audio,axis=1)
        else:
            info['audio'][index] = audio
        #info['audio'][index] = audio
    except Exception as e: # Some files are unable to be read by scipy's wavfile read, so skipping them and removing the corresponding rows
        info.drop(index,inplace=True)
        #print(e)
        
        print("Error, dropping row")
        print(f"size of info is now: {len(info)}")
info.reset_index(drop=True, inplace=True)
info.drop('index',axis=1,inplace=True)

<ipython-input-14-70eaf167f057>:3: WavFileWarning: Chunk (non-data) not understood, skipping it.
  [samplerate, x] = wavread(cAudioFilePath)
<ipython-input-18-771846dd385f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info['audio'][index] = np.mean(audio,axis=1)
<ipython-input-18-771846dd385f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info['audio'][index] = audio


Error, dropping row
size of info is now: 250
Error, dropping row
size of info is now: 249
Error, dropping row
size of info is now: 248
Error, dropping row
size of info is now: 247


In [19]:
def block_audio(x,blockSize,hopSize,fs):    
    # allocate memory    
    numBlocks = math.ceil(x.size / hopSize)    
    xb = np.zeros([numBlocks, blockSize])    
    # compute time stamps    
    t = (np.arange(0, numBlocks) * hopSize) / fs   
    x = np.concatenate((x, np.zeros(blockSize)),axis=0)    
    for n in range(0, numBlocks):        
        i_start = n * hopSize        
        i_stop = np.min([x.size - 1, i_start + blockSize - 1])        
        xb[n][np.arange(0,blockSize)] = x[np.arange(i_start, i_stop + 1)]    
    return (xb,t)

In [70]:
xb,t = block_audio(audio,2048,1024,44100)

In [20]:
def agg_mfccs(x):
    mfccs = librosa.feature.mfcc(x,n_mfcc = 13)
    mean = [np.mean(feature) for feature in mfccs]
    std = [np.std(feature) for feature in mfccs]
    mfcc_delta = librosa.feature.delta(mfccs)
    delta_mean=[np.mean(feature) for feature in mfcc_delta]
    delta_std=[np.std(feature) for feature in mfcc_delta]
    return mean,std,delta_mean,delta_std

In [22]:
def calc_stft(xb,fs=44100):
    stft = np.zeros((xb.shape[0],(int(xb[0].shape[0]/2)+1)))
    freqs = np.zeros((xb.shape[0],(int(xb[0].shape[0]/2)+1)))
    window = np.hanning(xb[0].shape[0])
    for i in range(xb.shape[0]):
        block= xb[i]
        # Apply Window to the block
        windowed_block = window * block 
        stft_blk = np.fft.fft(windowed_block)
        #stft_blk = np.fft.rfft(windowed_block)
        freq=np.fft.fftfreq(block.size,1/fs)
        freqs[i]=freq[:int(block.size/2)+1]
        stft_blk = np.abs(stft_blk)
        #stft_block = stft_blk[int((stft_blk.shape[0])/2):]
        stft_block = stft_blk[:int(((stft_blk.shape[0])/2)+1)]
        #stft_db = 10*np.log10(stft_block) # IS THIS CORRECT??
        stft[i]=stft_block#stft_db
    stft = np.array(stft)
    freqs=np.array(freqs)
    return stft,freqs

def extract_spectral_centroid(xb, fs):
    centroids = np.zeros(xb.shape[0])
    stft,freqs = calc_stft(xb,fs)
    #np.sum(magnitudes*freqs) / np.sum(magnitudes)
    for i in range(freqs.shape[0]):
        if np.all(stft[i] == 0):
            pass
        else:
            centroid = np.sum(stft[i]*freqs[i]) / np.sum(stft[i])
            centroids[i]=centroid
        #centroids.append(centroid)
    #centroids=np.array(centroids)
    return centroids
def calc_stft(xb,fs=44100):
    stft = np.zeros((xb.shape[0],(int(xb[0].shape[0]/2)+1)))
    freqs = np.zeros((xb.shape[0],(int(xb[0].shape[0]/2)+1)))
    window = np.hanning(xb[0].shape[0])
    for i in range(xb.shape[0]):
        block= xb[i]
        # Apply Window to the block
        windowed_block = window * block 
        stft_blk = np.fft.fft(windowed_block)
        #stft_blk = np.fft.rfft(windowed_block)
        freq=np.fft.fftfreq(block.size,1/fs)
        freqs[i]=freq[:int(block.size/2)+1]
        stft_blk = np.abs(stft_blk)
        #stft_block = stft_blk[int((stft_blk.shape[0])/2):]
        stft_block = stft_blk[:int(((stft_blk.shape[0])/2)+1)]
        #stft_db = 10*np.log10(stft_block) # IS THIS CORRECT??
        stft[i]=stft_block#stft_db
    stft = np.array(stft)
    freqs=np.array(freqs)
    return stft,freqs

def extract_spectral_centroid(xb, fs):
    centroids = np.zeros(xb.shape[0])
    stft,freqs = calc_stft(xb,fs)
    #np.sum(magnitudes*freqs) / np.sum(magnitudes)
    for i in range(freqs.shape[0]):
        if np.all(stft[i] == 0):
            pass
        else:
            centroid = np.sum(stft[i]*freqs[i]) / np.sum(stft[i])
            centroids[i]=centroid
        #centroids.append(centroid)
    #centroids=np.array(centroids)
    return centroids
def calc_stft(xb,fs=44100):
    stft = np.zeros((xb.shape[0],(int(xb[0].shape[0]/2)+1)))
    freqs = np.zeros((xb.shape[0],(int(xb[0].shape[0]/2)+1)))
    window = np.hanning(xb[0].shape[0])
    for i in range(xb.shape[0]):
        block= xb[i]
        # Apply Window to the block
        windowed_block = window * block 
        stft_blk = np.fft.fft(windowed_block)
        #stft_blk = np.fft.rfft(windowed_block)
        freq=np.fft.fftfreq(block.size,1/fs)
        freqs[i]=freq[:int(block.size/2)+1]
        stft_blk = np.abs(stft_blk)
        #stft_block = stft_blk[int((stft_blk.shape[0])/2):]
        stft_block = stft_blk[:int(((stft_blk.shape[0])/2)+1)]
        #stft_db = 10*np.log10(stft_block) # IS THIS CORRECT??
        stft[i]=stft_block#stft_db
    stft = np.array(stft)
    freqs=np.array(freqs)
    return stft,freqs

def extract_spectral_centroid(xb, fs):
    centroids = np.zeros(xb.shape[0])
    stft,freqs = calc_stft(xb,fs)
    #np.sum(magnitudes*freqs) / np.sum(magnitudes)
    for i in range(freqs.shape[0]):
        if np.all(stft[i] == 0):
            pass
        else:
            centroid = np.sum(stft[i]*freqs[i]) / np.sum(stft[i])
            centroids[i]=centroid
        #centroids.append(centroid)
    #centroids=np.array(centroids)
    return centroids

def extract_rms(xb):
    rms = np.zeros(xb.shape[0])
    for i in range(xb.shape[0]):
        if np.all(xb[i] == 0):
            r = 0.00001
        else:
            block = xb[i]
            r = np.sqrt(np.sum(block**2)/xb.shape[0])
            if r <= 0.00001: # Done to handle case when rms is 0 (for a block of all zeros
                r = 0.00001
            #rms.append(r)
        rms[i] = r
    #rms=np.array(rms)
    return 20*np.log10(rms)#rms,20*np.log10(rms) # What is reference value to convert to dB? Is it 1?

def extract_zerocrossingrate(xb):
    zcr= np.zeros(xb.shape[0])
    for i in range(xb.shape[0]):
        if np.all(xb[i] == 0):
            pass
        else:
            block = xb[i]
            zero_crossings = np.sum(np.abs(np.diff(np.sign(block)))) / block.shape[0] #np.nonzero(np.diff(block > 0))[0].size
            #zcr.append(zero_crossings)
            zcr[i] = zero_crossings 
    #zcr = np.array(zcr)
    return zcr

def extract_spectral_crest(xb):
    crest = np.zeros(xb.shape[0])
    stft,freqs = calc_stft(xb,44100)
    for i in range(stft.shape[0]):
        if np.all(stft[i] == 0):
            pass
        else:
            #crest.append((np.max(stft[i])/np.sum(stft[i])))
            crest[i] = np.max(stft[i])/np.sum(stft[i])
    #crest = np.array(crest)
    return crest

def extract_spectral_flux(xb):
    num_blocks=xb.shape[0] 
    blockSize = xb.shape[1]
    spectral_flux = np.zeros(num_blocks)
    stft,freqs = calc_stft(xb,44100)
    fft_len= stft.shape[1]
    n = 0
    k = 0
    for n in range(1,num_blocks):
        if np.all(stft[n] == 0):
            pass
        else:
            sum_flux = 0
            for k in range(fft_len):
                f = (abs(stft[n, k]) - abs(stft[n-1, k]))**2
                sum_flux += f
            flux = np.sqrt(sum_flux)/((blockSize/2)+1)
            spectral_flux[n] = flux #first flux value will be 0
    return spectral_flux


def extract_manual_features(x,fs):

    xb,ts = block_audio(x,2048,1024,fs)
    centroid = extract_spectral_centroid(xb,fs)
    rms = extract_rms(xb)
    zcr = extract_zerocrossingrate(xb)
    crest = extract_spectral_crest(xb)
    flux = extract_spectral_flux(xb)

    centroid_mean = np.mean(centroid)
    centroid_std = np.std(centroid)

    rms_mean = np.mean(rms)
    rms_std = np.std(rms)

    zcr_mean = np.mean(zcr)
    zcr_std = np.std(zcr)

    crest_mean = np.mean(crest)
    crest_std = np.std(crest)

    flux_mean = np.mean(flux)
    flux_std = np.std(flux)

    return centroid_mean, centroid_std, rms_mean, rms_std, zcr_mean, zcr_std, crest_mean, crest_std, flux_mean,flux_std

In [23]:
df=info
df['mfcc_mean'] = ''
df['mfcc_std'] = ''

df['delta_mfcc_mean'] = ''
df['delta_mfcc_std'] = ''

df['centroid_mean'] = ''
df['centroid_std'] = ''

df['rms_mean'] = ''
df['rms_std'] = ''

df['zcr_mean'] = ''
df['zcr_std'] = ''

df['crest_mean'] = ''
df['crest_std'] = ''

df['flux_mean'] = ''
df['flux_std'] = ''

for i in range(len(df)):
    #print(df['audio'][i].shape)
    mean,std,delta_mean,delta_std = agg_mfccs(df['audio'][i])
    # Extract 13 MFCCs - get mean and std deviation for each (26 features) + Delta MFCCs (26 features) = total 52 Features
    df['mfcc_mean'][i] = mean
    df['mfcc_std'][i] = std
    df['delta_mfcc_mean'][i] = delta_mean
    df['delta_mfcc_std'][i] = delta_std

    centroid_mean, centroid_std, rms_mean, rms_std, zcr_mean, zcr_std, crest_mean, crest_std, flux_mean,flux_std = extract_manual_features(df['audio'][i],44100)
    df['centroid_mean'][i] = centroid_mean
    df['centroid_std'][i] = centroid_std
    df['rms_mean'][i] = rms_mean
    df['rms_std'][i] = rms_std
    df['zcr_mean'][i] = zcr_mean
    df['zcr_std'][i] = zcr_std
    df['crest_mean'][i] = crest_mean
    df['crest_std'][i] = crest_std
    df['flux_mean'][i] = flux_mean
    df['flux_std'][i] = flux_std


df[['mfcc1_mean','mfcc2_mean','mfcc3_mean','mfcc4_mean','mfcc5_mean','mfcc6_mean','mfcc7_mean','mfcc8_mean','mfcc9_mean','mfcc10_mean','mfcc11_mean','mfcc12_mean','mfcc13_mean']]=pd.DataFrame(df.mfcc_mean.tolist(), index= df.index)
df[['mfcc1_std','mfcc2_std','mfcc3_std','mfcc4_std','mfcc5_std','mfcc6_std','mfcc7_std','mfcc8_std','mfcc9_std','mfcc10_std','mfcc11_std','mfcc12_std','mfcc13_std']]=pd.DataFrame(df.mfcc_std.tolist(), index= df.index)

df[['delta_mfcc1_mean','delta_mfcc2_mean','delta_mfcc3_mean','delta_mfcc4_mean','delta_mfcc5_mean','delta_mfcc6_mean','delta_mfcc7_mean','delta_mfcc8_mean','delta_mfcc9_mean','delta_mfcc10_mean','delta_mfcc11_mean','delta_mfcc12_mean','delta_mfcc13_mean']]=pd.DataFrame(df.delta_mfcc_mean.tolist(), index= df.index)
df[['delta_mfcc1_std','delta_mfcc2_std','delta_mfcc3_std','delta_mfcc4_std','delta_mfcc5_std','delta_mfcc6_std','delta_mfcc7_std','delta_mfcc8_std','delta_mfcc9_std','delta_mfcc10_std','delta_mfcc11_std','delta_mfcc12_std','delta_mfcc13_std']]=pd.DataFrame(df.delta_mfcc_std.tolist(), index= df.index)


<ipython-input-23-6309a878bdf4>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mfcc_mean'][i] = mean
<ipython-input-23-6309a878bdf4>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mfcc_std'][i] = std
<ipython-input-23-6309a878bdf4>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['delta_mfcc_mean'][i] = delta_mean
<ipython-input-23-6309a878bdf4>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [24]:
out = df.to_numpy()
np.save('./resources/working_data/data.npy', out)

In [25]:
df.columns

Index(['drummer', 'session', 'id', 'style', 'bpm', 'beat_type',
       'time_signature', 'midi_filename', 'audio_filename', 'duration',
       'split', 'audio', 'mfcc_mean', 'mfcc_std', 'delta_mfcc_mean',
       'delta_mfcc_std', 'centroid_mean', 'centroid_std', 'rms_mean',
       'rms_std', 'zcr_mean', 'zcr_std', 'crest_mean', 'crest_std',
       'flux_mean', 'flux_std', 'mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean',
       'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean',
       'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean', 'mfcc12_mean',
       'mfcc13_mean', 'mfcc1_std', 'mfcc2_std', 'mfcc3_std', 'mfcc4_std',
       'mfcc5_std', 'mfcc6_std', 'mfcc7_std', 'mfcc8_std', 'mfcc9_std',
       'mfcc10_std', 'mfcc11_std', 'mfcc12_std', 'mfcc13_std',
       'delta_mfcc1_mean', 'delta_mfcc2_mean', 'delta_mfcc3_mean',
       'delta_mfcc4_mean', 'delta_mfcc5_mean', 'delta_mfcc6_mean',
       'delta_mfcc7_mean', 'delta_mfcc8_mean', 'delta_mfcc9_mean',
       'delta_mfcc10_mean', 'd

# Load Feature file and classify

## Load Feature File

In [2]:
import os
import numpy as np
import scipy
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import matplotlib.pyplot as plt
import math
import librosa
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
import warnings
from sklearn.exceptions import DataConversionWarning
from sklearn.model_selection import RandomizedSearchCV
names=['drummer', 'session', 'id', 'style', 'bpm', 'beat_type',
       'time_signature', 'midi_filename', 'audio_filename', 'duration',
       'split', 'audio', 'mfcc_mean', 'mfcc_std', 'delta_mfcc_mean',
       'delta_mfcc_std', 'centroid_mean', 'centroid_std', 'rms_mean',
       'rms_std', 'zcr_mean', 'zcr_std', 'crest_mean', 'crest_std',
       'flux_mean', 'flux_std', 'mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean',
       'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean',
       'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean', 'mfcc12_mean',
       'mfcc13_mean', 'mfcc1_std', 'mfcc2_std', 'mfcc3_std', 'mfcc4_std',
       'mfcc5_std', 'mfcc6_std', 'mfcc7_std', 'mfcc8_std', 'mfcc9_std',
       'mfcc10_std', 'mfcc11_std', 'mfcc12_std', 'mfcc13_std',
       'delta_mfcc1_mean', 'delta_mfcc2_mean', 'delta_mfcc3_mean',
       'delta_mfcc4_mean', 'delta_mfcc5_mean', 'delta_mfcc6_mean',
       'delta_mfcc7_mean', 'delta_mfcc8_mean', 'delta_mfcc9_mean',
       'delta_mfcc10_mean', 'delta_mfcc11_mean', 'delta_mfcc12_mean',
       'delta_mfcc13_mean', 'delta_mfcc1_std', 'delta_mfcc2_std',
       'delta_mfcc3_std', 'delta_mfcc4_std', 'delta_mfcc5_std',
       'delta_mfcc6_std', 'delta_mfcc7_std', 'delta_mfcc8_std',
       'delta_mfcc9_std', 'delta_mfcc10_std', 'delta_mfcc11_std',
       'delta_mfcc12_std', 'delta_mfcc13_std']
df=pd.DataFrame(np.load('./resources/working_data/data.npy',allow_pickle=True),columns=names)

### Feature Normalization and train-test split

In [4]:
X = df[['mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean',
       'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean',
       'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean', 'mfcc12_mean',
       'mfcc13_mean', 'mfcc1_std', 'mfcc2_std', 'mfcc3_std', 'mfcc4_std',
       'mfcc5_std', 'mfcc6_std', 'mfcc7_std', 'mfcc8_std', 'mfcc9_std',
       'mfcc10_std', 'mfcc11_std', 'mfcc12_std', 'mfcc13_std',
       'delta_mfcc1_mean', 'delta_mfcc2_mean', 'delta_mfcc3_mean',
       'delta_mfcc4_mean', 'delta_mfcc5_mean', 'delta_mfcc6_mean',
       'delta_mfcc7_mean', 'delta_mfcc8_mean', 'delta_mfcc9_mean',
       'delta_mfcc10_mean', 'delta_mfcc11_mean', 'delta_mfcc12_mean',
       'delta_mfcc13_mean', 'delta_mfcc1_std', 'delta_mfcc2_std',
       'delta_mfcc3_std', 'delta_mfcc4_std', 'delta_mfcc5_std',
       'delta_mfcc6_std', 'delta_mfcc7_std', 'delta_mfcc8_std',
       'delta_mfcc9_std', 'delta_mfcc10_std', 'delta_mfcc11_std',
       'delta_mfcc12_std', 'delta_mfcc13_std',\
       'centroid_mean','centroid_std','rms_mean','rms_std','zcr_mean','zcr_std','crest_mean','crest_std','flux_mean','flux_std']]
y=df[['style']]

for col in X.columns:
    #df[col] = df[col]/max(np.abs(df[col]))
    mean = np.mean(X[col])
    std = np.std(X[col])
    X[col] = (X[col] - mean)/std # z-score normalization

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

<ipython-input-4-feaf435abdcc>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = (X[col] - mean)/std # z-score normalization


In [5]:
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings('ignore', category=UndefinedMetricWarning) 
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

## Results with MFCCs + Delta MFCCs

In [126]:
print("-------------------------------------------------------------------------------")
print("            KNN with k=4              ")
print("-------------------------------------------------------------------------------")
k=4
KNN_model = KNeighborsClassifier(n_neighbors=k)
KNN_model.fit(X_train, y_train)
KNN_prediction = KNN_model.predict(X_test)
score=precision_score(y_test, KNN_prediction, average='macro')
accuracy = accuracy_score(y_test, KNN_prediction)
cm = confusion_matrix(y_test, KNN_prediction) # clean, highfry, layered, lowfry, midfry, no vocals
print(f"Confusion Matrix:\n {cm}")
print(f"Accuracy score: {accuracy}")
print(f"Macro accuracy score: {score}")

print("-------------------------------------------------------------------------------")
print("            SVM with C=1, kernel = rbf, degree = 3              ")
print("-------------------------------------------------------------------------------")

SVM_model = make_pipeline(StandardScaler(), SVC(gamma='auto',C=1.0, kernel='rbf', degree=3,random_state=None))
SVM_model.fit(X_train, y_train)
SVM_prediction = SVM_model.predict(X_test)
score=precision_score(y_test, SVM_prediction, average='macro')
accuracy = accuracy_score(y_test, SVM_prediction)
cm = confusion_matrix(y_test, SVM_prediction) # clean, highfry, layered, lowfry, midfry, no vocals
print(f"Confusion Matrix:\n {cm}")
print(f"Accuracy score: {accuracy}")
print(f"Macro accuracy score: {score}")

print("-------------------------------------------------------------------------------")
print("            RF with n_estimators = 100, criterion=gini, max_depth=None, min_samples_split=2,min_samples_leaf=1,max_features='auto', max_leaf_nodes=None,class_weight='balanced'              ")
print("-------------------------------------------------------------------------------")

RF_model = RandomForestClassifier(n_estimators=100,criterion='gini',max_depth=None,\
    min_samples_split=2,min_samples_leaf=1,max_features='auto',max_leaf_nodes=None,class_weight='balanced')
RF_model.fit(X_train, y_train)
RF_prediction = RF_model.predict(X_test)
score=precision_score(y_test, RF_prediction, average='macro')
accuracy = accuracy_score(y_test, RF_prediction)
cm = confusion_matrix(y_test, RF_prediction) # clean, highfry, layered, lowfry, midfry, no vocals
print(f"Confusion Matrix:\n {cm}")
print(f"Accuracy score: {accuracy}")
print(f"Macro accuracy score: {score}")

-------------------------------------------------------------------------------
            KNN with k=4              
-------------------------------------------------------------------------------
Confusion Matrix:
 [[ 9  0  0  2]
 [ 0  7  0  2]
 [ 0  0  2  8]
 [ 4  8  0 40]]
Accuracy score: 0.7073170731707317
Macro accuracy score: 0.732051282051282
-------------------------------------------------------------------------------
            SVM with C=1, kernel = rbf, degree = 3              
-------------------------------------------------------------------------------
Confusion Matrix:
 [[ 0  0  0 11]
 [ 0  0  0  9]
 [ 0  0  1  9]
 [ 0  0  0 52]]
Accuracy score: 0.6463414634146342
Macro accuracy score: 0.4104938271604938
-------------------------------------------------------------------------------
            RF with n_estimators = 100, criterion=gini, max_depth=None, min_samples_split=2,min_samples_leaf=1,max_features='auto', max_leaf_nodes=None,class_weight='balanced'          

## Overall Hyperparameter Optimization

In [217]:
print("-------------------------------------------------------------------------------")
print("            KNN             ")
print("{'n_neighbors': 9}")
print("-------------------------------------------------------------------------------")
k= [int(x) for x in range(4,10)]
knn_grid = {'n_neighbors':k}


knn = KNeighborsClassifier()
KNN_model=RandomizedSearchCV(estimator = knn, param_distributions=knn_grid,cv=10,verbose=2,n_jobs=4)
KNN_model.fit(X_train, y_train)
print(KNN_model.best_params_)
KNN_prediction = KNN_model.predict(X_test)
score=precision_score(y_test, KNN_prediction, average='macro')
accuracy = accuracy_score(y_test, KNN_prediction)
cm = confusion_matrix(y_test, KNN_prediction) # clean, highfry, layered, lowfry, midfry, no vocals
print(f"Confusion Matrix:\n {cm}")
print(f"Accuracy score: {accuracy}")
print(f"Macro accuracy score: {score}")

print("-------------------------------------------------------------------------------")
print("            SVM             ")
print("{'kernel': 'poly', 'gamma': 'scale', 'degree': 5, 'decision_function_shape': 'ovr', 'coef0': 3.2, 'C': 1.053646986645436}")
print("-------------------------------------------------------------------------------")

C = [x**0.01 for x in range(25,200)]
kernel = ['rbf','linear','poly','sigmoid']
degree = [int(x) for x in range(1,8)]
gamma = ['scale','auto']
coef0 = [x*0.1 for x in range(0,40)]
decision_function_shape = ['ovo','ovr']

svm_grid = { 'C' : C,
            'kernel' : kernel,
            'degree' : degree,
            'gamma' : gamma,
            'coef0' : coef0,
            'decision_function_shape' : decision_function_shape
            }

svc = SVC(random_state=0)


svc_RandomGrid = RandomizedSearchCV(estimator = svc, param_distributions=svm_grid,cv=10,verbose=2,n_jobs=4)
svc_RandomGrid.fit(X_train,y_train)
print(svc_RandomGrid.best_params_)
y_prediction = svc_RandomGrid.predict(X_test)
score=precision_score(y_test, y_prediction, average='macro')
accuracy = accuracy_score(y_test, y_prediction)
cm = confusion_matrix(y_test, y_prediction) # clean, highfry, layered, lowfry, midfry, no vocals
print(f"Confusion Matrix:\n {cm}")
print(f"Accuracy score: {accuracy}")
print(f"Macro accuracy score: {score}")

print("-------------------------------------------------------------------------------")
print("            Random Forest             ")
print("{'n_estimators': 650, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_leaf_nodes': 14, 'max_features': 30, 'max_depth': 46, 'criterion': 'entropy', 'class_weight': 'balanced_subsample'}")
print("-------------------------------------------------------------------------------")

n_estimators= [x for x in range(10,1000,10)]
criterion=['gini','entropy']

max_depth=[y for y in range(1,1000,5)]
max_depth.append(None)

min_samples_split=[x for x in range(2,10)]
min_samples_leaf=[x for x in range(1,10)]
max_features=[x for x in range(0,100)]
max_features.append('auto')
max_features.append('sqrt')
max_features.append('log2')
max_leaf_nodes=[x for x in range(0,100)]
class_weight=['balanced','balanced_subsample']


rf_grid = { 'n_estimators' : n_estimators,
            'criterion' : criterion,
            'max_depth': max_depth,
            'min_samples_split': min_samples_split,
            'min_samples_leaf': min_samples_leaf,
            'max_features': max_features,
            'max_leaf_nodes': max_leaf_nodes,
            'class_weight': class_weight 
            }

rf = RandomForestClassifier(random_state=0)
RF_model = RandomizedSearchCV(estimator = rf, param_distributions=rf_grid,cv=10,verbose=2,n_jobs=4)
RF_model.fit(X_train, y_train)
print(RF_model.best_params_)
RF_prediction = RF_model.predict(X_test)
score=precision_score(y_test, RF_prediction, average='macro')
accuracy = accuracy_score(y_test, RF_prediction)
cm = confusion_matrix(y_test, RF_prediction) # clean, highfry, layered, lowfry, midfry, no vocals
print(f"Confusion Matrix:\n {cm}")
print(f"Accuracy score: {accuracy}")
print(f"Macro accuracy score: {score}")

-------------------------------------------------------------------------------
            KNN             
{'n_neighbors': 9}
-------------------------------------------------------------------------------
Fitting 10 folds for each of 6 candidates, totalling 60 fits


/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


{'n_neighbors': 9}
Confusion Matrix:
 [[ 4  0  0  8]
 [ 1  1  0  3]
 [ 0  0  0 10]
 [ 3  4  0 48]]
Accuracy score: 0.6463414634146342
Macro accuracy score: 0.34891304347826085
-------------------------------------------------------------------------------
            SVM             
{'kernel': 'poly', 'gamma': 'scale', 'degree': 5, 'decision_function_shape': 'ovr', 'coef0': 3.2, 'C': 1.053646986645436}
-------------------------------------------------------------------------------
Fitting 10 folds for each of 10 candidates, totalling 100 fits
{'kernel': 'rbf', 'gamma': 'scale', 'degree': 7, 'decision_function_shape': 'ovo', 'coef0': 0.9, 'C': 1.052637571345141}
Confusion Matrix:
 [[ 0  0  0 12]
 [ 0  1  0  4]
 [ 0  0  0 10]
 [ 0  0  0 55]]
Accuracy score: 0.6829268292682927
Macro accuracy score: 0.41975308641975306
-------------------------------------------------------------------------------
            Random Forest             
{'n_estimators': 650, 'min_samples_split': 5, 'min_sa

/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.69558824 0.73235294 0.74448529        nan 0.69669118 0.64227941
        nan 0.66617647        nan 0.59926471]
  warnings.warn(


{'n_estimators': 960, 'min_samples_split': 4, 'min_samples_leaf': 4, 'max_leaf_nodes': 86, 'max_features': 27, 'max_depth': 426, 'criterion': 'entropy', 'class_weight': 'balanced'}
Confusion Matrix:
 [[ 2  2  0  8]
 [ 0  2  0  3]
 [ 0  0  5  5]
 [ 2  1  1 51]]
Accuracy score: 0.7317073170731707
Macro accuracy score: 0.62363184079602


# Hyperparameter tuning

## Best Results:



RF:  
accuracy                                                     0.743902  
macro_accuracy                                               0.678571  
{'n_estimators': 800,
 'min_samples_split': 8,
 'min_samples_leaf': 2,
 'max_leaf_nodes': 29,
 'max_features': 45,
 'max_depth': 796,
 'criterion': 'entropy',
 'class_weight': 'balanced'}


SVM: 

accuracy 0.8658536585365854  
macro-accuracy 0.8566308243727598  
[[ 5  1  0  3]
 [ 0  8  0  3]
 [ 0  0  4  2]
 [ 0  0  2 54]]
{'kernel': 'poly', 'gamma': 'auto', 'degree': 4, 'decision_function_shape': 'ovr', 'coef0': 0.9, 'C': 1.051659640188661}

accuracy                                                     0.8414634146341463  
macro_accuracy                                               0.8418898809523809  
{'kernel': 'poly', 'gamma': 'scale', 'degree': 3, 'decision_function_shape': 'ovr', 'coef0': 1.1, 'C': 1.053359910724275}

accuracy                                                     0.780488  
macro_accuracy                                               0.728571  
{'kernel': 'poly',
 'gamma': 'scale',
 'degree': 3,
 'decision_function_shape': 'ovr',
 'coef0': 0.7000000000000001,
 'C': 1.053475672085955}

accuracy              0.792683  
macro_accuracy        0.768056  
 {'kernel': 'poly',
 'gamma': 'scale',
 'degree': 7,
 'decision_function_shape': 'ovr',
 'coef0': 0.6000000000000001,
 'C': 1.053646986645436}


### SVM Params

In [31]:
C = [x**0.01 for x in range(25,200)]
kernel = ['rbf','linear','poly','sigmoid']
degree = [int(x) for x in range(1,8)]
gamma = ['scale','auto']
coef0 = [x*0.1 for x in range(0,40)]
decision_function_shape = ['ovo','ovr']

svm_grid = { 'C' : C,
            'kernel' : kernel,
            'degree' : degree,
            'gamma' : gamma,
            'coef0' : coef0,
            'decision_function_shape' : decision_function_shape
            }

svc = SVC()
svm_df=pd.DataFrame()
svm_df['trial']=[]
svm_df['accuracy'] = []
svm_df['macro_accuracy'] = []
svm_df['confusion_matrix']=[]
svm_df['params']=[]

trial=[]
accuracy=[]
macro_accuracy=[]
confusion=[]
params=[]
for i in range(0,50):
    trial.append(i)
    svc_RandomGrid = RandomizedSearchCV(estimator = svc, param_distributions=svm_grid,cv=10,verbose=0,n_jobs=4) #verbose=2
    svc_RandomGrid.fit(X_train,y_train)
    #print(svc_RandomGrid.best_params_)
    params.append(svc_RandomGrid.best_params_)
    y_prediction = svc_RandomGrid.predict(X_test)
    score=precision_score(y_test, y_prediction, average='macro')
    acc = accuracy_score(y_test, y_prediction)
    cm = confusion_matrix(y_test, y_prediction) # clean, highfry, layered, lowfry, midfry, no vocals

    accuracy.append(acc)
    macro_accuracy.append(score)
    confusion.append(cm)
    #print(f"Confusion Matrix:\n {cm}")
    #print(f"Accuracy score: {accuracy}")
    #print(f"Macro accuracy score: {score}")


svm_df['trial']= trial
svm_df['accuracy'] = accuracy
svm_df['macro_accuracy'] = macro_accuracy
svm_df['confusion_matrix']=confusion
svm_df['params']=params

svm_df['macro_accuracy'].max()

0.8566308243727598

In [32]:
svm_df[svm_df['macro_accuracy'] == svm_df['macro_accuracy'].max() ]

,trial,accuracy,macro_accuracy,confusion_matrix,params
37,37,0.865854,0.856631,"[[5, 1, 0, 3], [0, 8, 0, 3], [0, 0, 4, 2], [0,...","{'kernel': 'poly', 'gamma': 'auto', 'degree': ..."


In [34]:
for val in svm_df.loc[37]:
    print(val)

37
0.8658536585365854
0.8566308243727598
[[ 5  1  0  3]
 [ 0  8  0  3]
 [ 0  0  4  2]
 [ 0  0  2 54]]
{'kernel': 'poly', 'gamma': 'auto', 'degree': 4, 'decision_function_shape': 'ovr', 'coef0': 0.9, 'C': 1.051659640188661}


In [19]:
df.loc[44]

trial                                                              44
accuracy                                                     0.792683
macro_accuracy                                               0.768056
confusion_matrix    [[2, 0, 0, 5], [0, 2, 0, 5], [1, 0, 3, 4], [0,...
params              {'kernel': 'poly', 'gamma': 'scale', 'degree':...
Name: 44, dtype: object

In [9]:
df.loc[5]

trial                                                               5
accuracy                                                     0.780488
macro_accuracy                                               0.728571
confusion_matrix    [[3, 0, 0, 4], [1, 1, 0, 5], [1, 0, 3, 4], [0,...
params              {'kernel': 'poly', 'gamma': 'scale', 'degree':...
Name: 5, dtype: object

Current best results:

SVM:  
accuracy                                                     0.780488  
macro_accuracy                                               0.728571  
{'kernel': 'poly',
 'gamma': 'scale',
 'degree': 3,
 'decision_function_shape': 'ovr',
 'coef0': 0.7000000000000001,
 'C': 1.053475672085955}

accuracy              0.792683  
macro_accuracy        0.768056  
 {'kernel': 'poly',
 'gamma': 'scale',
 'degree': 7,
 'decision_function_shape': 'ovr',
 'coef0': 0.6000000000000001,
 'C': 1.053646986645436}

### Random Forest Tuning

In [21]:
n_estimators= [x for x in range(10,1000,10)]
criterion=['gini','entropy']

max_depth=[y for y in range(1,1000,5)]
max_depth.append(None)

min_samples_split=[x for x in range(2,10)]
min_samples_leaf=[x for x in range(1,10)]
max_features=[x for x in range(0,100)]
max_features.append('auto')
max_features.append('sqrt')
max_features.append('log2')
max_leaf_nodes=[x for x in range(0,100)]
class_weight=['balanced','balanced_subsample']
rf_grid = { 'n_estimators' : n_estimators,
            'criterion' : criterion,
            'max_depth': max_depth,
            'min_samples_split': min_samples_split,
            'min_samples_leaf': min_samples_leaf,
            'max_features': max_features,
            'max_leaf_nodes': max_leaf_nodes,
            'class_weight': class_weight 
            }
rf_df=pd.DataFrame()
rf_df['trial']=[]
rf_df['accuracy'] = []
rf_df['macro_accuracy'] = []
rf_df['confusion_matrix']=[]
rf_df['params']=[]

trial=[]
accuracy=[]
macro_accuracy=[]
confusion=[]
params=[]

rf = RandomForestClassifier(random_state=0)
RF_model = RandomizedSearchCV(estimator = rf, param_distributions=rf_grid,cv=10,verbose=2,n_jobs=4)
for i in range(0,200):
    trial.append(i)
    RF_model.fit(X_train, y_train)
    #print(RF_model.best_params_)
    params.append(RF_model.best_params_)
    RF_prediction = RF_model.predict(X_test)
    score=precision_score(y_test, RF_prediction, average='macro')
    acc = accuracy_score(y_test, RF_prediction)
    cm = confusion_matrix(y_test, RF_prediction) # clean, highfry, layered, lowfry, midfry, no vocals

    accuracy.append(acc)
    macro_accuracy.append(score)
    confusion.append(cm)
    #print(f"Confusion Matrix:\n {cm}")
    #print(f"Accuracy score: {accuracy}")
    #print(f"Macro accuracy score: {score}")

rf_df['trial']= trial
rf_df['accuracy'] = accuracy
rf_df['macro_accuracy'] =  macro_accuracy
rf_df['confusion_matrix']= confusion
rf_df['params']= params
rf_df['macro_accuracy'].max()

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.41617647        nan 0.67316176        nan 0.62389706 0.62463235
 0.75735294 0.65992647 0.58823529 0.73308824]
  warnings.warn(


Fitting 10 folds for each of 10 candidates, totalling 100 fits


/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.67904412        nan        nan 0.71507353
 0.69117647 0.72095588 0.61139706 0.73308824]
  warnings.warn(


Fitting 10 folds for each of 10 candidates, totalling 100 fits


...

Fitting 10 folds for each of 10 candidates, totalling 100 fits


/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.72169118        nan        nan        nan
        nan 0.72242647 0.72720588 0.75183824]
  warnings.warn(


Fitting 10 folds for each of 10 candidates, totalling 100 fits


/usr/local/lib/python3.9/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan 0.73272059        nan        nan
        nan        nan        nan 0.70257353]
  warnings.warn(


0.6785714285714286

In [24]:
rf_df[rf_df['macro_accuracy'] == 0.6785714285714286]

,trial,accuracy,macro_accuracy,confusion_matrix,params
1,1,0.743902,0.678571,"[[1, 1, 0, 5], [0, 3, 0, 4], [0, 0, 2, 6], [0,...","{'n_estimators': 800, 'min_samples_split': 8, ..."


In [25]:
rf_df.loc[1]

trial                                                               1
accuracy                                                     0.743902
macro_accuracy                                               0.678571
confusion_matrix    [[1, 1, 0, 5], [0, 3, 0, 4], [0, 0, 2, 6], [0,...
params              {'n_estimators': 800, 'min_samples_split': 8, ...
Name: 1, dtype: object

In [26]:
rf_df.loc[1]['params']

{'n_estimators': 800,
 'min_samples_split': 8,
 'min_samples_leaf': 2,
 'max_leaf_nodes': 29,
 'max_features': 45,
 'max_depth': 796,
 'criterion': 'entropy',
 'class_weight': 'balanced'}

Current best results:

RF:  
accuracy                                                     0.743902  
macro_accuracy                                               0.678571  
{'n_estimators': 800,
 'min_samples_split': 8,
 'min_samples_leaf': 2,
 'max_leaf_nodes': 29,
 'max_features': 45,
 'max_depth': 796,
 'criterion': 'entropy',
 'class_weight': 'balanced'}


# Optimized hyperparameters

In [20]:
svm_params = {'kernel': 'poly',
 'gamma': 'scale',
 'degree': 3,
 'decision_function_shape': 'ovr',
 'coef0': 0.7000000000000001,
 'C': 1.053475672085955,
 'random_state':0}


rf_params = {'n_estimators': 800,
 'min_samples_split': 8,
 'min_samples_leaf': 2,
 'max_leaf_nodes': 29,
 'max_features': 45,
 'max_depth': 796,
 'criterion': 'entropy',
 'class_weight': 'balanced'}

print("-------------------------------------------------------------------------------")
print("            KNN             ")
print("-------------------------------------------------------------------------------")
#k= [int(x) for x in range(4,10)]
#knn_grid = {'n_neighbors':k}


KNN_model = KNeighborsClassifier(n_neighbors=7)
#KNN_model=RandomizedSearchCV(estimator = knn, param_distributions=knn_grid,cv=10,verbose=2,n_jobs=4)
KNN_model.fit(X_train, y_train)
#print(KNN_model.best_params_)
KNN_prediction = KNN_model.predict(X_test)
score=precision_score(y_test, KNN_prediction, average='macro')
accuracy = accuracy_score(y_test, KNN_prediction)
cm = confusion_matrix(y_test, KNN_prediction) # clean, highfry, layered, lowfry, midfry, no vocals
print(f"Confusion Matrix:\n {cm}")
print(f"Accuracy score: {accuracy}")
print(f"Macro accuracy score: {score}")

print("-------------------------------------------------------------------------------")
print("            SVM             ")
print("-------------------------------------------------------------------------------")
SVM_model = make_pipeline(StandardScaler(), SVC(kernel='poly',gamma='scale',degree=3,decision_function_shape='ovr',coef0=0.7000000000000001,C=1.053475672085955,random_state=0))
SVM_model.fit(X_train, y_train)
SVM_prediction = SVM_model.predict(X_test)
score=precision_score(y_test, SVM_prediction, average='macro')
cm = confusion_matrix(y_test, SVM_prediction) # clean, highfry, layered, lowfry, midfry, no vocals
acc = accuracy_score(y_test, SVM_prediction)
print(cm)
print("Accuracy Score: ",acc)
print("Macro accuracy Score:",score)

print("-------------------------------------------------------------------------------")
print("            RF              ")
print("-------------------------------------------------------------------------------")

RF_model = RandomForestClassifier(n_estimators=800,min_samples_split = 8, min_samples_leaf = 2,max_leaf_nodes = 29, max_depth = 796, criterion = 'entropy',class_weight='balanced',random_state=0)
RF_model.fit(X_train, y_train)
RF_prediction = RF_model.predict(X_test)
score=precision_score(y_test, RF_prediction, average='macro')
cm = confusion_matrix(y_test, RF_prediction) # clean, highfry, layered, lowfry, midfry, no vocals
acc = accuracy_score(y_test, RF_prediction)
print(cm)
print("Accuracy Score: ",acc)
print("Macro accuracy Score:",score)

-------------------------------------------------------------------------------
            KNN             
-------------------------------------------------------------------------------
Confusion Matrix:
 [[ 7  0  0  2]
 [ 1  5  0  5]
 [ 0  0  0  6]
 [ 2  3  1 50]]
Accuracy score: 0.7560975609756098
Macro accuracy score: 0.5296626984126984
-------------------------------------------------------------------------------
            SVM             
-------------------------------------------------------------------------------
[[ 4  1  0  4]
 [ 0  3  0  8]
 [ 0  0  5  1]
 [ 0  0  2 54]]
Accuracy Score:  0.8048780487804879
Macro accuracy Score: 0.8175639658848615
-------------------------------------------------------------------------------
            RF              
-------------------------------------------------------------------------------
[[ 6  0  0  3]
 [ 0  3  0  8]
 [ 0  0  3  3]
 [ 1  3  1 51]]
Accuracy Score:  0.7682926829268293
Macro accuracy Score: 0.7229395604395604


# Old Unused code:

In [44]:
# def freq_to_mel(freq):
#     return 2595.0 * np.log10(1.0 + freq / 700.0)

# def mel_to_freq(mels):
#     return 700.0 * (10.0**(mels / 2595.0) - 1.0)

# def get_filter_points(fmin, fmax, mel_filter_num, FFT_size, fs=44100):
#     fmin_mel = freq_to_mel(fmin)
#     fmax_mel = freq_to_mel(fmax)
#     mels = np.linspace(fmin_mel, fmax_mel, num=mel_filter_num+2)
#     freqs = mel_to_freq(mels)
    
#     return np.floor((FFT_size + 1) / fs * freqs).astype(int), freqs

# def get_filters(filter_points, FFT_size):
#     filters = np.zeros((len(filter_points)-2,int(FFT_size/2+1)))
    
#     for n in range(len(filter_points)-2):
#         filters[n, filter_points[n] : filter_points[n + 1]] = np.linspace(0, 1, filter_points[n + 1] - filter_points[n])
#         filters[n, filter_points[n + 1] : filter_points[n + 2]] = np.linspace(1, 0, filter_points[n + 2] - filter_points[n + 1])
    
#     return filters
    
# def dct(dct_filter_num, filter_len):
#     basis = np.empty((dct_filter_num,filter_len))
#     basis[0, :] = 1.0 / np.sqrt(filter_len)
    
#     samples = np.arange(1, 2 * filter_len, 2) * np.pi / (2.0 * filter_len)

#     for i in range(1, dct_filter_num):
#         basis[i, :] = np.cos(i * samples) * np.sqrt(2.0 / filter_len)
        
#     return basis

In [75]:
# n_mfccs=13

# freq_min = 0
# freq_high = fs / 2
# fft_size=xb[0].shape[0]
# dct_filter_num = 40
# dct_filters = dct(dct_filter_num, n_mfccs)
# window = get_window("hann",fft_size , fftbins=True)

# audio_winT = np.transpose(xb*window)

# audio_fft = np.empty((int(1 + fft_size // 2), audio_winT.shape[1]), dtype=np.complex64, order='F')

# for n in range(audio_fft.shape[1]):
#     audio_fft[:, n] = fft.fft(audio_winT[:, n], axis=0)[:audio_fft.shape[0]]

# audio_fft = np.transpose(audio_fft)
# audio_power = np.square(np.abs(audio_fft))
# filter_points, mel_freqs = get_filter_points(freq_min, freq_high, n_mfccs, fft_size, fs)
# filters = get_filters(filter_points, fft_size)

# enorm = 2.0 / (mel_freqs[2:n_mfccs+2] - mel_freqs[:n_mfccs])
# filters *= enorm[:, np.newaxis]

# audio_filtered = np.dot(filters, np.transpose(audio_power))
# audio_log = 10.0 * np.log10(audio_filtered)

# dct_filters = dct(dct_filter_num, n_mfccs)

# cepstral_coefficents = scipy.fftpack.dct(audio_log)#np.dot(dct_filters, audio_log)


<ipython-input-75-42337085eaf8>:26: RuntimeWarning: divide by zero encountered in log10
  audio_log = 10.0 * np.log10(audio_filtered)


In [66]:
# def calc_mfccs(xb,fs=44100,n_mfccs=13):
#     freq_min = 0
#     freq_high = fs / 2
#     fft_size=xb[0].shape[0]
#     dct_filter_num = 40
#     dct_filters = dct(dct_filter_num, n_mfccs)
#     window = get_window("hann",fft_size , fftbins=True)
#     mfccs=[]
#     for block in xb:
#         # Apply Window to the block
#         windowed_block = window * block
#         stft_blk = np.fft.fft(windowed_block)[:windowed_block.shape[0]]
#         #stft_blk = np.fft.rfft(windowed_block)
#         stft_blk = np.square(np.abs(stft_blk)) #POWER SPECTRUM NOT MAGNITUDE
#         #stft_block = stft_blk[int((stft_blk.shape[0])/2):]
#         filter_points, mel_freqs = get_filter_points(freq_min, freq_high, n_mfccs, fft_size, fs)
#         filters = get_filters(filter_points, fft_size)
#         enorm = 2.0 / (mel_freqs[2:n_mfccs+2] - mel_freqs[:n_mfccs])
#         filters *= enorm[:, np.newaxis]
#         audio_filtered = np.dot(filters, np.transpose(stft_blk))
#         audio_log = 10.0 * np.log10(audio_filtered)
#         cepstral_coefficents = np.dot(dct_filters, audio_log)
#         mfccs.append(cepstral_coefficents)
#     return mfccs

In [69]:
window = get_window("hann",fft_size , fftbins=True)
windowed_block = window * block
stft_blk = np.fft.fft(windowed_block)
#stft_blk = np.fft.rfft(windowed_block)
stft_blk = np.square(np.abs(stft_blk)) #POWER SPECTRUM NOT MAGNITUDE
#stft_block = stft_blk[int((stft_blk.shape[0])/2):]
filter_points, mel_freqs = get_filter_points(freq_min, freq_high, n_mfccs, fft_size, fs)
filters = get_filters(filter_points, fft_size)
enorm = 2.0 / (mel_freqs[2:n_mfccs+2] - mel_freqs[:n_mfccs])
filters *= enorm[:, np.newaxis]
audio_filtered = np.dot(filters, np.transpose(stft_blk))
audio_log = 10.0 * np.log10(audio_filtered)
cepstral_coefficents = np.dot(dct_filters, audio_log)

ValueError: shapes (13,1025) and (2048,5294) not aligned: 1025 (dim 1) != 2048 (dim 0)

In [47]:

calc_mfccs(xb,44100,13)


ValueError: shapes (13,1025) and (2048,) not aligned: 1025 (dim 1) != 2048 (dim 0)

In [ ]:
def agg_mfccs(x):
    mfccs = librosa.feature.mfcc(x,n_mfcc = 13)
    mean = [np.mean(feature) for feature in mfccs]
    std = [np.std(feature) for feature in mfccs]
    mfcc_delta = librosa.feature.delta(mfccs)
    delta_mean=[np.mean(feature) for feature in mfcc_delta]
    delta_std=[np.std(feature) for feature in mfcc_delta]
    return mean,std,delta_mean,delta_std